# Lab 02: Stance Detection with Logistic Regression and Word Embeddings using scikit-learn

In this lab session we will implement a very simple Logistic Regression Classifier for Stance Detection using https://scikit-learn.org

Stance detection consists of classifying a given document as expressing an AGAINST, FAVOR or NEUTRAL attitude/stance with respect to a given topic. In this particular lab, we use the Task A data from the Semeval 2016 Twitter dataset for Stance detection: https://alt.qcri.org/semeval2016/task6/ 

Scikit-learn allows you to quickly experiment with a large number of machine learning algorithms in low resource environments (in comparison to neural network approaches). Scikit-learn also provides a large number of functionalities to process data and evaluate and visualize the obtained results.

Unlike other toolkits we will see during the course, scikit-learn is a library with an easy to use API ideal for quick experimentation with a large variety of models and algorithms. Usually, it is a good starting point for classification tasks.


## ADD DOCUMENTATION

+ TODO: Add inline documentation to the notebook cells, this way you can learn what it does.


## Functions for loading and pre-processing the corpus

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import nltk
import numpy as np

nltk.download('stopwords')

# load data
'''
Loads a series of TSV files to a single Pandas DataFrame
'''
def load_data(fnames):
    data = []
    for fname in fnames:
        data.append(pd.read_csv(fname, sep='\t', encoding='utf-8'))
    data = pd.concat(data)
    targets = set(data['Target'])
    return data, list(targets)

def tokenized_tweets(df):
    """Function to tokenize the tweets using the tokenizer from NLTK, 
    it converts the tweets to a DF wz a column wz the nu tweets"""
    tknzr = nltk.TweetTokenizer()
    df['Tokenized_tweet'] = df['Tweet'].apply(tknzr.tokenize)
    return df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# ASSIGNMENT 1

+ TODO: describe how the function loading the word embeddings works. What is the input, the output and how it is obtained. HINT: Inspect the embedding glove twitter lexicon used in Assignment 2.

In [3]:
from nltk.corpus import stopwords
import string
from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer

def preprocess(data, tokenize=True, remove_stopwords=True, remove_none=True):
    '''
    This function have three parts:

    The first one is tokenize: 
        This function evoques "tokenized_tweets" from NLTK's Tweet tokenizer to
        cleane the text and store it to a new column of a DataFrame called 
        "Clean_tweet". The first function will make also a new column called 
        "Tokenized_tweet" with the clean tokenized tweets.
    The second one is remove_stopwords:
        Based on the NLTK stop words, this function will remove the punctuation 
        marks and all stop words from tweets.
    The third one is remove_none:
        In the column "Stance", we have the opinions of the topics annotated with
        three labels - AGAINST, FAVOR and NONE. The NONE ones are not rellevant 
        for the ML algorith, so we eliminate them with this function.
        '''

    if tokenize:
        data = tokenized_tweets(data)
        data['Clean_tweet'] = data['Tokenized_tweet']
    if remove_stopwords:
        stop = stopwords.words('english')
        data['Clean_tweet'] = data['Clean_tweet'].apply(lambda sentence: [word for word in sentence if word not in stop])
        data['Clean_tweet'] = data['Clean_tweet'].apply(lambda sentence: [word for word in sentence if not all([c in string.punctuation for c in word])])
    if remove_none:
        data = data[data['Stance'] != 'NONE']
    return data[['Target','Clean_tweet', 'Stance']]   
    
def read_glove(path, dim, skipFirstRow=False):
   
    if skipFirstRow == False:
      df = pd.read_csv(path, sep=" ", quoting=3, header=None, index_col=0)
    else:
      df = pd.read_csv(path, sep=" ", quoting=3,  header=None, skiprows = [0], 
                       index_col=0) 
    glove = {key: val.values for key, val in df.T.items()}
    
    return glove

 ##############################################################################
 ## TODO: provide a description of this function. HINT: it helps to look     ##
 ## at the glove twitter word embeddings                                     ##
 ##############################################################################

def gloveVectorize(glove, text):
   
    dim = len(glove["the"]) 
    X = np.zeros( (len(text), dim) )
    
    for text_id, t in enumerate(text):
        
        tmp = np.zeros((1, dim))

        # remove oov words

        words = [w for w in t if w in glove.keys()]
        for word in words:
            tmp[:] += glove[word]

        if len(words) == 0:
            X[text_id, :] = np.zeros((1, dim)) 
        
        else:
            X[text_id, :] = tmp/len(words)
    
    return X

def encode_labels(labels):
    enc = preprocessing.LabelEncoder()
    encoded = enc.fit_transform(labels)
    decoded = enc.inverse_transform(encoded)
    return encoded, decoded

def data_as_numpy(data):
    return np.asarray(data['Clean_tweet']), np.asarray(data['Stance'])


In [4]:
# TASK A in-target supervised: 
# load train / test
trial_file = "/content/drive/MyDrive/2022-ILTAPP/datasets/stance-semeval2016/semeval2016-task6-trialdata.utf-8.txt"
train_file = "/content/drive/MyDrive/2022-ILTAPP/datasets/stance-semeval2016/semeval2016-task6-trainingdata.utf-8.txt"
test_file = "/content/drive/MyDrive/2022-ILTAPP/datasets/stance-semeval2016/SemEval2016-Task6-subtaskA-testdata-gold.txt"

training_data, targets = load_data([train_file, trial_file])
test_data, _ = load_data([test_file])

# show original training data
display(training_data)

#training_data[training_data['Target'] == targets[0]].head()

# preprocess
training_preproc_data = preprocess(training_data, remove_none=False) #Preprocess the dataset, DO NOT get rid of Tweets whose stance is "NONE"
test_preproc_data = preprocess(test_data, remove_none=False )

# show clean training data
display(training_preproc_data)

,ID,Target,Tweet,Stance
0,101,Atheism,dear lord thank u for all of ur blessings forg...,AGAINST
1,102,Atheism,"Blessed are the peacemakers, for they shall be...",AGAINST
2,103,Atheism,I am not conformed to this world. I am transfo...,AGAINST
3,104,Atheism,Salah should be prayed with #focus and #unders...,AGAINST
4,105,Atheism,And stay in your houses and do not display you...,AGAINST
...,...,...,...,...
95,96,Legalization of Abortion,@Corey_Frizzell @PEILiberalParty and most Isla...,NONE
96,97,Legalization of Abortion,@Docjp Pressure? It's their job and they are f...,NONE
97,98,Legalization of Abortion,I love how #liberals only accuse #conservative...,AGAINST
98,99,Legalization of Abortion,Help your friend figure out how they're going ...,NONE


,Target,Clean_tweet,Stance
0,Atheism,"[dear, lord, thank, u, ur, blessings, forgive,...",AGAINST
1,Atheism,"[Blessed, peacemakers, shall, called, children...",AGAINST
2,Atheism,"[I, conformed, world, I, transformed, renewing...",AGAINST
3,Atheism,"[Salah, prayed, #focus, #understanding, #Allah...",AGAINST
4,Atheism,"[And, stay, houses, display, like, times, igno...",AGAINST
...,...,...,...
95,Legalization of Abortion,"[@Corey_Frizzell, @PEILiberalParty, Islanders,...",NONE
96,Legalization of Abortion,"[@Docjp, Pressure, It's, job, failing, miserab...",NONE
97,Legalization of Abortion,"[I, love, #liberals, accuse, #conservatives, w...",AGAINST
98,Legalization of Abortion,"[Help, friend, figure, they're, going, get, st...",NONE


# ASSIGNMENT 2

+ TODO try with different word embeddings. For example:
  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz (1.2GB)


In [5]:
#TODO try with the different embeddings in resources directory and see which one obtains better results

# set path for word embeddings and vectorize
pretrained_wv_path = '/content/drive/MyDrive/2022-ILTAPP/resources/glove.twitter.27B.25d.txt.gz'

glove = read_glove(pretrained_wv_path, 300)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def fit_lr(train_x, train_y, c=1.0, weights='equal'):
    if weights == 'equal':
        logreg = Pipeline([
            ("scaler", StandardScaler()),
            ("logit", LogisticRegression(C=c, solver="lbfgs", max_iter=1000))
        ])
    else:
        logreg = Pipeline([
        ("scaler", StandardScaler()),
        ("logit", LogisticRegression(C=c, solver="lbfgs", class_weight=weights, max_iter=1000))
        ])
    logreg.fit(train_x, train_y)
    return logreg

# ASSIGNMENT 3

+ TODO: evaluate accuracy, F1 macro, F1 micro using sklearn functions. HINT: Check previous labs.

**NOTE**: The results seen below are with GloVe embeddings (glove.twitter.27B.25d.txt.gz). Results with another embedding model, "FastText", is available in a code cell below this one. We have marked the cell that has the results with FastText in another area of the code.

**RESULTS WITH GLOVE EMBEDDINGS**

In [7]:
best_c = 1.2
class_weight = 'balanced'

# prediction map
label_map = {0: 'AGAINST',
             1: 'FAVOR',
             2: 'NONE'}

predictions = pd.DataFrame()

training_texts, training_labels = data_as_numpy(training_preproc_data)
train_x = gloveVectorize(glove, training_texts)
train_y, labels = encode_labels(training_labels)

test_texts, test_labels = data_as_numpy(test_preproc_data)
test_x = gloveVectorize(glove, test_texts)
test_y, labels = encode_labels(test_labels)

# fit model
logreg = fit_lr(train_x, train_y, c=best_c, weights=class_weight)

# predict
stance = logreg.predict(test_x)
stance_probs = logreg.predict_proba(test_x)

# TODO evaluate accuracy, F1 macro, F1 micro using sklearn functions

predictions = test_data[['ID', 'Target', 'Tweet']]
predictions_probs = test_data[['ID', 'Target', 'Tweet']]
predictions['Stance'] = [label_map[s] for s in stance]
predictions_probs['Stance'] = [s for s in stance_probs]

predictions = predictions.sort_values(by='ID')
predictions_probs = predictions_probs.sort_values(by='ID')

display(predictions)

#Accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, stance)
print("\nAccuracy:", accuracy)

#F1 macro
from sklearn.metrics import f1_score
f1_macro = f1_score(test_y, stance, average='macro')
print("F1 macro:", f1_macro)

#F1 micro
f1_macro = f1_score(test_y, stance, average='micro')
print("F1 macro:", f1_macro)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ID,Target,Tweet,Stance
0,10001,Atheism,He who exalts himself shall be humbled; a...,AGAINST
1,10002,Atheism,RT @prayerbullets: I remove Nehushtan -previou...,NONE
2,10003,Atheism,@Brainman365 @heidtjj @BenjaminLives I have so...,AGAINST
3,10004,Atheism,#God is utterly powerless without Human interv...,FAVOR
4,10005,Atheism,@David_Cameron Miracles of #Multiculturalism...,NONE
...,...,...,...,...
1244,11245,Legalization of Abortion,@MetalheadMonty @tom_six I followed him before...,AGAINST
1245,11246,Legalization of Abortion,"For he who avenges blood remembers, he does no...",AGAINST
1246,11247,Legalization of Abortion,Life is sacred on all levels. Abortion does no...,AGAINST
1247,11248,Legalization of Abortion,"@ravensymone U refer to ""WE"" which =""YOU"" & a ...",FAVOR



Accuracy: 0.45396317053642915
F1 macro: 0.42987547140876714
F1 macro: 0.45396317053642915


**RESULTS WITH cc.en.300.vec.gz EMBEDDINGS CAN BE SEEN IN THE CODE CELL BELOW**



In [8]:

FastText_path = '/content/drive/MyDrive/2022-ILTAPP/resources/cc.en.300.vec.gz'
FastText = read_glove(FastText_path, 800, skipFirstRow=True) 
#the function is self explaining, skip the first row

best_c = 1.2
class_weight = 'balanced'

# prediction map
label_map = {0: 'AGAINST',
             1: 'FAVOR',
             2: 'NONE'}

predictions = pd.DataFrame()

training_texts, training_labels = data_as_numpy(training_preproc_data)
train_x = gloveVectorize(FastText, training_texts)
train_y, labels = encode_labels(training_labels)

test_texts, test_labels = data_as_numpy(test_preproc_data)
test_x = gloveVectorize(FastText, test_texts)
test_y, labels = encode_labels(test_labels)

# fit model
logreg = fit_lr(train_x, train_y, c=best_c, weights=class_weight)

# predict
stance = logreg.predict(test_x)
stance_probs = logreg.predict_proba(test_x)

# TODO evaluate accuracy, F1 macro, F1 micro using sklearn functions

predictions = test_data[['ID', 'Target', 'Tweet']]
predictions_probs = test_data[['ID', 'Target', 'Tweet']]
predictions['Stance'] = [label_map[s] for s in stance]
predictions_probs['Stance'] = [s for s in stance_probs]

predictions = predictions.sort_values(by='ID')
predictions_probs = predictions_probs.sort_values(by='ID')

display(predictions)

#Accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, stance)
print("\nAccuracy:", accuracy)

#F1 macro
from sklearn.metrics import f1_score
f1_macro = f1_score(test_y, stance, average='macro')
print("F1 macro:", f1_macro)

#F1 micro
f1_macro = f1_score(test_y, stance, average='micro')
print("F1 macro:", f1_macro)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ID,Target,Tweet,Stance
0,10001,Atheism,He who exalts himself shall be humbled; a...,AGAINST
1,10002,Atheism,RT @prayerbullets: I remove Nehushtan -previou...,AGAINST
2,10003,Atheism,@Brainman365 @heidtjj @BenjaminLives I have so...,AGAINST
3,10004,Atheism,#God is utterly powerless without Human interv...,FAVOR
4,10005,Atheism,@David_Cameron Miracles of #Multiculturalism...,AGAINST
...,...,...,...,...
1244,11245,Legalization of Abortion,@MetalheadMonty @tom_six I followed him before...,NONE
1245,11246,Legalization of Abortion,"For he who avenges blood remembers, he does no...",AGAINST
1246,11247,Legalization of Abortion,Life is sacred on all levels. Abortion does no...,AGAINST
1247,11248,Legalization of Abortion,"@ravensymone U refer to ""WE"" which =""YOU"" & a ...",NONE



Accuracy: 0.5164131305044035
F1 macro: 0.48752662103887817
F1 macro: 0.5164131305044035


# (BONUS) ASSIGNMENT 4

Check the Feature-based lab and perform the following steps:

+ TODO: Change code from above to use SVM instead of logistic regression.
+ TODO modify code to run over every target in the dataframes for training and test.

Write a table to:
+ TODO: Compare results with logistic regression and SVM using word embeddings on the test set.
+ TODO: Compare results with feature-based SVM on the test set.

In [9]:
import sklearn
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def fit_svm(train_x, train_y, c=1.0, weights='equal'):
    if weights == 'equal':
        svm = Pipeline([
            ("scaler", StandardScaler()),
            ("logit", SVC(C=c,probability=True, max_iter=1000))
        ])
    else:
        svm = Pipeline([
        ("scaler", StandardScaler()),
        ("logit", SVC(C=c, probability=True, class_weight=weights, max_iter=1000))
        ])
    svm.fit(train_x, train_y)
    return svm

best_c = 1.2
class_weight = 'balanced'

# prediction map
label_map = {0: 'AGAINST',
             1: 'FAVOR',
             2: 'NONE'}

predictions = pd.DataFrame()

training_texts, training_labels = data_as_numpy(training_preproc_data)
train_x = gloveVectorize(glove, training_texts)
train_y, labels = encode_labels(training_labels)

test_texts, test_labels = data_as_numpy(test_preproc_data)
test_x = gloveVectorize(glove, test_texts)
test_y, labels = encode_labels(test_labels)

# fit model
svm = fit_svm(train_x, train_y, c=best_c, weights=class_weight)

# predict
stance = svm.predict(test_x)
stance_probs = svm.predict_proba(test_x)

# TODO evaluate accuracy, F1 macro, F1 micro using sklearn functions

# Accuracy
acc_svm = sklearn.metrics.accuracy_score(test_y, stance)

# F1 macro
F1macro_svm = sklearn.metrics.f1_score(test_y, stance, average='macro')

# F1 micro
F1micro_svm = sklearn.metrics.f1_score(test_y, stance, average='micro')



predictions = test_data[['ID', 'Target', 'Tweet']]
predictions_probs = test_data[['ID', 'Target', 'Tweet']]
predictions['Stance'] = [label_map[s] for s in stance]
predictions_probs['Stance'] = [s for s in stance_probs]

predictions = predictions.sort_values(by='ID')
predictions_probs = predictions_probs.sort_values(by='ID')

display(predictions)

print(test_y)
print(stance)

print(f"Accuracy = {acc_svm}\tMacro = {F1macro_svm}\tMicro = {F1micro_svm}")

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ID,Target,Tweet,Stance
0,10001,Atheism,He who exalts himself shall be humbled; a...,NONE
1,10002,Atheism,RT @prayerbullets: I remove Nehushtan -previou...,NONE
2,10003,Atheism,@Brainman365 @heidtjj @BenjaminLives I have so...,AGAINST
3,10004,Atheism,#God is utterly powerless without Human interv...,FAVOR
4,10005,Atheism,@David_Cameron Miracles of #Multiculturalism...,NONE
...,...,...,...,...
1244,11245,Legalization of Abortion,@MetalheadMonty @tom_six I followed him before...,NONE
1245,11246,Legalization of Abortion,"For he who avenges blood remembers, he does no...",AGAINST
1246,11247,Legalization of Abortion,Life is sacred on all levels. Abortion does no...,FAVOR
1247,11248,Legalization of Abortion,"@ravensymone U refer to ""WE"" which =""YOU"" & a ...",NONE


[0 0 0 ... 0 0 0]
[2 2 0 ... 1 2 2]
Accuracy = 0.3586869495596477	Macro = 0.36025438590595305	Micro = 0.3586869495596477
